## What is a neural network?

Neural network is a collection of neurons that are connected by layers. Each neuron is small computing until that performs simple calculations to collectively solve a problem. They are organized in layers. There are 3 types of layers: 
* input layer
* hidden layer
* output layer

Each layer contains a number of neurons, except for the input layer. Neural networks mimic the way a human brain processes information.

### Components of a neural network

#### Activation function
`Activation function determines whether a neuron should be activated or not`. The computations that happen in a neural network include applying an activation function. `If a neuron activates, then it means the input is important`. They are different kinds of activation functions. `The choice of which activation funciton to use dependes on what you want the output to be`. Another important role of an activation function is to `add non-linearity to the model`.
  * *Binary* used to set an output node to 1 if function result is positive and 0 if the function result is negative

$$
f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ 1, & \text{if } x\geq 0\\ \end{cases}}
$$

  * *Sigmod* is used to predict the probability of an output node being between 0 and 1

$$
f(x) = {\large \frac{1}{1+e^{-x}}}
$$

  * *Tanh* is used to predict the probability of an output node being between -1 and 1. Used in classification use cases.

$$
f(x) = {\large \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}}
$$

  * *ReLU(Rectified Linear Unit)* used to set the output node to 0 if function result is negative and keeps the result value if the result is a postive values.

$$
f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ x, & \text{if } x\geq 0\\ \end{cases}}
$$

#### Weights
`Weights` influence how well the output of our network will come close to the expected output value. As an input enter the neuron, it gets multiplied by a weight value and the resulting output is either observed, or passed to the next layer in the neural network. Weights for all neurons in a layer are organize into one tensor.

#### Bias
`Bias` makes up the difference between the activation function's output and its intended output. A low bias suggest that the network is making more assumptions about the form of the output, whereas a high bias value makes less assumptions about the form of the output.


We can say that output y of a neural nework layer with weights W and bias b is computed as summation of the inputs multiply by the weights plus the bias

$$
x = \sum{(weights * inputs) + bias}
$$

where f(x) is the activation function.

## Build a neural network

Neural networks are comprised of layer/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we will build a neural network to classify images in the FashionMNIST dataset.

pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
%matplotlib.inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get a hardware device for training

In [ ]:
# check the paltform, Apple Silicon or Linux
import os, platform

torch_device="cpu"

if 'kaggle' in os.environ.get('KAGGLE_URL_BASE','localhost'):
    torch_device = 'cuda'
else:
    torch_device = 'mps' if platform.system() == 'Darwin' else 'cpu'

os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

torch_device

### Define the class

We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

Our neural network are composed of the following:

* The input layer with 28x28 or 784 features/pixels
* The first linear module takes the input 784 features and transforms it to a hidden layer with 512 features
* The ReLU activation function will be applied in the transformation
* The second linear module take 512 features as input from the first hidden layer and transforms it to the next hidden layer with 512 features
* The ReLU activation function will be applied in the transformation
* The third linear module take 512 features as input form the second hidden layer and transforms it  to the output layer with 10, which is the number of classes
* The ReLU activation function will be applied in the transformation


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequantial(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self,x):
        x =self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Move the Neural Network to the device we defined earlier and print the model's architecture.

In [ ]:
model = NeuralNetwork().to(torch_device)
print(model)

To use the model, we pass it the input data. This executes the models `forward`, along with some background operation. However, do not call `model.forward()` directly. Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class. We get the prediction densities by passing it through an instance of the `nn.Softmax`.

In [ ]:
X =torch.rand(1,28,28, device=torch_device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

### Weight and Bias

The `nn.linear` module randomly initialized the *weights* and *bias* for each layer and internally stores the values in Tensors.

In [ ]:
print(f'First Linear weights: {model.linear_relu_stack[0].weight} \n')
print(f'First Linear bias: {model.linear_relu_stack[0].bias} \n')

### Model layers

Let's break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [ ]:
input_sample = torch.rand(3,28,28)
print(input_sample.size())

### nn.Flatten

We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained). Each of the pixels are pass to the input layer of the neural network.

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_sample)
print(flat_image.size())

### nn.Linear

The linear layer is a module that applies a linear transformation on the input using it's stored weights and biases. The gayscale value of each pixel in the input layer will be connected to neurons in the hidden layer for calculation. The calculation used for the transformation is `weight` * `input` + `bias`.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

### nn.ReLU

Non-linear activations are what create the complex mappings between the model's input and output. They are applied after linear tranformations to introduce `nonlinearity`, helping neural networks learn a wide variety of phenomena. In this model, we use `nn.ReLU` between our linear layers, but there's other activations to introduce non-linearity in your model.

The ReLU activation function takes the output from the linear layer calculation and replaces the negative values with zeros.

Linear output:
$$
{ x = {weight * input + bias}}
$$
ReLU:

$$
f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ x, & \text{if } x\geq 0\\ \end{cases}}
$$

In [ ]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

### nn.Sequential

`nn.Sequential` is an ordered container of modules. The data is passed through all the modulers in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`.

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1, nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax

The last linear layer of the neural network returns `logits` - raw values in [-infty, infty], which are passed to the `nn.Softmax` module. The Softmax activation function is used to calcualte the probability of the output from the neural network. It is only used on output layer  of a neural network. the results are scaled to values [0,1] representing the model's predicted densities for each class. `dim` parameter indicates the dimension along which the result values must sum to 1. The node with the highest probability predicts the desired output.

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model parameters

Many layers inside a neural network are `parameterized`, i.e. have associated weights and biases that are optimized during training. Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model's `parameters()` or `named_parameters()` methods.

In [ ]:
print('Model structure:', model, '\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n')